## Работа с данными в pytorch

https://pytorch.org/docs/stable/data.html
Обычно работа с данными декомпозирована на два класса:
    
### `torch.utils.data.Dataset`

Класс для работы с семплами. Сюда часто добавляют логику скачивания датасета, препроцессинг и аугментации.

Для работы со своими данными нужно отнаследоваться от этого класса и реализовать два метода: `__len__` и `__getitem__`.
Сначала мы воспользуемся готовым датасетом из [`torchvision.datasets`](https://pytorch.org/docs/stable/torchvision/datasets.html)

### `torch.utils.data.Dataloader`

Загрузчик данных, загружает семплы из Dataset, занимается семплирование, батчеванием, перемешиванием и т.д.
Умеет в multiprocessing, это необходимо при работе со сколько-нибудь большими датасетами.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from IPython.display import clear_output

import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

from tqdm import tqdm_notebook as tqdm
from collections import defaultdict

# папку для загрузки можно поменять
download_path = '/tmp'
mnist_train = datasets.MNIST(download_path, train=True, download=True, transform=transforms.ToTensor())
mnist_val = datasets.MNIST(download_path, train=False, download=True, transform=transforms.ToTensor())

**Задание 0. (0.1 балла)**
1. В каком виде возвращает семплы итератор по `mnist_train`?
2. Отобразите несколько примеров

In [ ]:
# напишите ответ текстом или кодом здесь
<your code here>

In [ ]:
# обязательно прсмотрите на то, в каком виде возвращаются семплы
plt.figure(figsize=[6, 6])
for i in range(4):
    plt.subplot(2, 2, i + 1)
    # get img and label from mnist_train    
    img, label = <your code here>

    plt.title("Label: {}".format(label))
    plt.imshow(img, cmap='gray')

In [ ]:
def plot_history(log, name=None):
    if name is None:
        name='loss'
    train_points, val_points = [], []
    train_key = 'train_{}'.format(name)
    val_key = 'val_{}'.format(name)

    for entry in log:
        if train_key in entry:
            train_points.append((entry['train_step'], entry[train_key]))
        if val_key in entry:
            val_points.append((entry['train_step'], entry[val_key]))
    
    plt.figure()
    plt.title(name)
    x, y = list(zip(*train_points))
    plt.plot(x, y, label='train', zorder=1)
    x, y = list(zip(*val_points))
    plt.scatter(x, y, label='val', zorder=2, marker='+', s=180, c='orange')
    
    plt.legend(loc='best')
    plt.grid()
    plt.show()
    

def train_model(model, optimizer, train_dataset, val_dataset, batch_size=32, epochs=10):
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    
    log = []
    train_step = 0
    for epoch in range(epochs):
        model.train()
        for x, y in tqdm(train_loader):
            <your code here>
            acc = ...
            loss = ...
            
            log.append(dict(
                train_loss=loss,
                train_acc=acc,
                train_step=train_step,
            ))
            train_step += 1

        tmp = defaultdict(list)
        model.eval()
        for x, y in tqdm(val_loader):
            with torch.no_grad():
                <your code here>
                acc = ...
                loss = ...
                
                tmp['acc'].append(acc)
                tmp['loss'].append(loss
                
        log.append(dict(
            val_loss = np.mean(tmp['loss']),  # скаляры
            val_acc = np.concatenate(tmp['acc']).mean(),  # массивы, возможно разной длины
            train_step=train_step,
        ))
        
        clear_output()
        plot_history(log, name='loss')
        plot_history(log, name='acc')

## Сверточные сети

Мы рассмотрим сверточные сети на примере MNIST, заодно поучимся пользоваться стандартными pytorch-классами для работы с данными.

В случае картинок, обычно работают с входными тензорами размера `[batch_size, channels, height, widht]` (такой порядок осей называется channels-first или NCHW).

Сверточные сети обычно собираются из последовательности слоев:

### Convolution
https://pytorch.org/docs/stable/nn.html#convolution-layers

По тензору бежит скользящее окно и в нем вычисляется свертка с ядром.
Обычно говорят о пространственных размерах сверток, например 1x1 или 3x3  свертки, подразумевая, что ядра имеют размер `[1,1,ch]` или `[3,3,ch]`.

Сейчас часто используются чуть более сложные варианты сверток: 
- dilated (atrous, дырявые), 
- depth-wise
- pointwise
- separable
- group


### Pooling
https://pytorch.org/docs/stable/nn.html#pooling-layers

Действуют аналогично свертках, но не имеют весов, а в бегущем окне вычисляется какая-нибудь функция, например max или mean.


### Global pooling (Adaptive Pooling)
https://pytorch.org/docs/stable/nn.html#adaptivemaxpool1d

Глобальные пулинги (в pytorch адаптивные) убирают пространственные размерности, превращая `[bs, ch, h, w]` в `[bs, ch, 1, 1]`.

#### Задание 1 (0.2 балла)

1. Реализуйте сверточную сеть, 2xConv+ReLU+MPooling + Dense.
**Hint: Воспользуйтесь оберткой `nn.Sequential`**

2. Натренируйте модель с помощью функции train_model.

In [ ]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        <your code here>
        
    def forward(self, x):
        <your code here>
        return <something>

In [ ]:
# pytorch предоставляет ряд методов для обхода весов в моделях
# так можно посчитать количество обучаемых параметров, или построить изображение вычислительного графа
def count_parameters(model):
    model_parameters = filter(lambda p: p.requires_grad, model.parameters())
    return sum([np.prod(p.size()) for p in model_parameters])

model = ConvNet()
print("Total number of trainable parameters:", count_parameters(model))

In [ ]:
# Ошибка классификации после обучения должна быть ниже 1.5%
opt = torch.optim.SGD(model.parameters(), lr=1e-3)
train_model(model, opt, mnist_train, mnist_val)

# Затухающие и взрывающиеся градиенты

Продолжаем экспериментировать с MNIST. 
В этом разделе нас будут интересовать особенности обучения глубоких сетей.

1. Напишите свою функцию train_model, которая помимо графиков acc/loss будет подсчитывать нормы градиентов на каждом тренировочном шаге для кажого из обучаемых слоев.

2. Напишите класс для построения сеток с произвольным количеством слоев и произвольными активациями

3. Проведите ряд экспериментов для сверточной сети и для сетей с полносвязными слоями.



**Hint: вам может пригодиться `model.named_parameters()` чтобы обойти слои модели**

In [ ]:
def plot_grads(grad_log):
    buffers = defaultdict(list)
    
    for entry in grad_log:
        for k, v in entry.items():
            buffers[k].append(v)
    
    names_to_plot = sorted(set(buffers.keys()).difference({'train_step'}))
    steps = buffers['train_step']
    
    plt.figure()
    plt.title('grads')
    
    for i, name in enumerate(names_to_plot):
        plt.semilogy(
            buffers[name], label=name, 
            color=plt.cm.coolwarm(i / len(names_to_plot)),
        )    
    
    plt.legend(loc='best')
    plt.grid()
    plt.show()


def train_model(model, optimizer, train_dataset, val_dataset, batch_size=32, epochs=10):
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    
    grad_log, log = [], []
    train_step = 0
    for epoch in range(epochs):
        model.train()
        for x, y in tqdm(train_loader):
            <your code>
            entry = {}
            entry['train_step'] = train_step
            grad_log.append(entry)
            
            log.append(dict(
                train_loss=...,
                train_acc=...,
                train_step=train_step,
            ))
            train_step += 1

        tmp = defaultdict(list)
        model.eval()
        for x, y in tqdm(val_loader):
            with torch.no_grad():
                <your code>
                acc = ...
                loss = ...
                tmp['acc'].append(acc)
                tmp['loss'].append(loss
                
        log.append(dict(
            val_loss = np.mean(tmp['loss']),  # скаляры
            val_acc = np.concatenate(tmp['acc']).mean(),  # массивы, возможно разной длины
            train_step=train_step,
        ))
        
        clear_output()
        plot_history(log, name='loss')
        plot_history(log, name='acc')
        plot_grads(grad_log)

In [ ]:
# проверим на сверточной сети
model = ConvNet()
opt = torch.optim.SGD(model.parameters(), lr=1e-3)
x = train_model(model, opt, mnist_train, mnist_val)

**Задание 3 (0.2)** Реализуйте построение сети с произвольным числом (>1) полносвязных слоев с задаваемой функцией активации

In [ ]:
class DenseNet(nn.Module):
    def __init__(self, num_layers, hidden_size, activation):
        super().__init__()
        <your code here>
        
    def forward(self, x):
        <your code here>
        return <something>

**Задание 4 (0.3 балла)**  Проведите ряд экспериментов с градиентами для 10 слоев и функций активаций {Sigmoid, ReLU}.

Hidden size можно взять 10.

In [ ]:
# 10 слоев по 10, Sigmoid
model = DenseNet(...)
opt = torch.optim.SGD(model.parameters(), lr=1e-3)
x = train_model(model, opt, mnist_train, mnist_val)

In [ ]:
# 10 слоев по 10, ReLU
model = DenseNet(...)
opt = torch.optim.SGD(model.parameters(), lr=1e-3)
x = train_model(model, opt, mnist_train, mnist_val)

**Задание 5(0.2 балла)** Добавьте skip-connections и проверьте как протекают градиенты для 20 слоев и Sigmoid функции активации.

In [ ]:
class DenseResNet(nn.Module):
    def __init__(self, num_layers, hidden_size, activation):
        super().__init__()
        <your code here>
        
    def forward(self, x):
        <your code here>
        return <something>

In [ ]:
# 10 слоев по 10, Sigmoid
model = DenseResNet(...)
opt = torch.optim.SGD(model.parameters(), lr=1e-3)
x = train_model(model, opt, mnist_train, mnist_val)